In [9]:
from itertools import chain

from IPython.core.debugger import prompt
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from nltk.sem.chat80 import items
from sympy.physics.units import temperature

load_dotenv()
chat = ChatOpenAI(temperature=0.1)

template = PromptTemplate.from_template("What is the distance between {country_a} and {country_b}.")
template.format(country_a="Mexico", country_b="Thailand")

chat.predict(prompt)

'Hello! How can I assist you today?'

In [12]:
template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geopraphy expert. And you only reply in {language}."),
        ("ai", "Ciao, mi chiamo {name}!"),
        ("human", "What is the distance between {country_a} and {country_b}. Also, what is your name?")
    ]
)
prompt = template.format_messages(
    language="Greek",
    name="Socrates",
    country_a="Mexico",
    country_b="Thailand"

)

chat.predict_messages(prompt)

AIMessage(content='Γεια σας! Η απόσταση μεταξύ του Μεξικού και της Ταϊλάνδης είναι περίπου 16.000 χιλιόμετρα. Το όνομά μου είναι Σωκράτης. Πώς μπορώ να βοηθήσω;')

In [16]:
from langchain.schema import BaseOutputParser


class CommaOutputParser(BaseOutputParser):

    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))


p = CommaOutputParser()
p.parse("Hello, how, are, you")

['Hello', 'how', 'are', 'you']

In [21]:
template = ChatPromptTemplate.from_messages([
    ("system",
     "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items}. Do NOT reply with anything else."),
    ("human", "{question}")
])

chain = template | chat | CommaOutputParser()
chain.invoke({"max_items": 5, "question": "What are the pokemons?"})

['Pikachu', 'Charmander', 'Bulbasaur', 'Squirtle', 'Jigglypuff']

In [9]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from dotenv import load_dotenv
from langchain.callbacks import StreamingStdOutCallbackHandler

load_dotenv()
chat = ChatOpenAI(temperature=0.1, streaming=True,callbacks=[])
chef_prompt = ChatPromptTemplate.from_messages([
    ("system",
     "You are a world-class international chef. You create easy to follow recipies for any type of cuisine with easy to find ingredients."),
    ("human", "I want to cook {cuisine} food.")
])

chef_chain = chef_prompt | chat

In [10]:
veg_chef_prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        "You are a vegetarian chef specialized on making traditional recipies vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it.",
    ),
    ("human", "{recipe}"),
])

veg_chain = veg_chef_prompt | chat

final_chain = {"recipe": chef_chain} | veg_chain

final_chain.invoke({
    "cuisine": "indian"
})

AIMessageChunk(content="To make a vegetarian version of Chicken Tikka Masala, we can replace the chicken with a plant-based alternative such as tofu or paneer. Here's how you can adapt the recipe:\n\nIngredients:\n- 1 lb firm tofu or paneer, cut into bite-sized pieces\n- 1 cup plain yogurt (you can use dairy-free yogurt for a vegan version)\n- 2 tbsp lemon juice\n- 2 tsp ground cumin\n- 2 tsp paprika\n- 1 tsp ground turmeric\n- 1 tsp garam masala\n- 1 tsp ground coriander\n- 1/2 tsp cayenne pepper (adjust to taste)\n- Salt and pepper to taste\n- 2 tbsp vegetable oil\n- 1 onion, diced\n- 3 cloves garlic, minced\n- 1-inch piece of ginger, grated\n- 1 can (14 oz) crushed tomatoes\n- 1 cup coconut cream (or dairy-free heavy cream alternative)\n- Fresh cilantro, chopped (for garnish)\n- Cooked rice or naan bread (for serving)\n\nInstructions:\n1. Follow the same marinating process as the original recipe, but use tofu or paneer instead of chicken. Tofu can be marinated for at least 30 minute